In [2]:
import cv2
import numpy as np

from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model

In [3]:
prototxt_path = "./face detection/deploy.prototxt.txt"
weights_path = "./face detection/res10_300x300_ssd_iter_140000.caffemodel"

detector = cv2.dnn.readNet(prototxt_path, weights_path)

CNN = load_model("./checkpoint")

In [4]:
with_mask_paths = ["./data/with_mask/with_mask_" + str(i) + ".jpg" for i in range(1, 3725 + 1)]
without_mask_paths = ["./data/without_mask/without_mask_" + str(i)  + ".jpg" for i in range(1, 3828 + 1)]
'''
image = []

for path in np.concatenate((with_mask_paths, without_mask_paths)):
    image.append(cv2.imread(path))
'''

test_image = cv2.imread("./testset/2.jpg")

In [5]:
blob = cv2.dnn.blobFromImage(test_image, 1.0, (300, 300),
                             (104.0, 177.0, 123.0))
(h, w) = test_image.shape[:2]

detector.setInput(blob)
detections = detector.forward()

In [7]:
threshold = 0.15

for i in range(0, detections.shape[2]):
    confidence = detections[0, 0, i, 2]

    if confidence > threshold:
        box = detections[0, 0, i, 3:7] * np.array([w,h,w,h])
        (start_x, start_y, end_x, end_y) = box.astype("int")

        (start_x, start_y) = (max(0, start_x), max(0, start_y))
        (end_x, end_y) = (min(w - 1, end_x), min(h - 1, end_y))

        face = test_image[start_y:end_y, start_x:end_x]
        face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
        face = cv2. resize(face, (224, 224))
        face = img_to_array(face)
        face = preprocess_input(face)
        face = np.expand_dims(face, axis = 0)

        (without_mask, mask) = CNN.predict(face)[0]

        label = "mask" if mask > without_mask else "no mask"
        color = (0, 255, 0) if label == "mask" else (0, 0, 255)

        label = "{}: {:.2f}%".format(label, max(mask, without_mask) * 100)

        cv2.putText(test_image, label, (start_x, start_y - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
        cv2.rectangle(test_image, (start_x, start_y), (end_x, end_y), color, 2)

cv2.imshow("output", test_image)
cv2.waitKey(0)

27